In [16]:
import os
import json
import sqlite3

import pandas as pd

In [17]:
filepath = 'data/'

In [33]:
data = []

for filename in os.listdir(filepath):
    cnx = sqlite3.connect(filepath + filename)
    df_string = pd.read_sql_query("SELECT * FROM COUNTERS_STRING_TIME_DATA", cnx)
    data.append(pd.DataFrame(df_string))

In [34]:
df = pd.concat(data, ignore_index = True)

In [36]:
df

,MEASUREMENT_TIME,ID_INPUT,VALUE,PRIVATE_DATA
0,2023-01-15 18:20:31.552,3,esrv.exe,0
1,2023-01-15 18:20:31.552,4,VsDebugConsole.exe,0
2,2023-01-15 18:20:31.552,5,ConsoleWindowClass,0
3,2023-01-15 18:20:33.574,3,Downloads,0
4,2023-01-15 18:20:33.575,4,explorer.exe,0
...,...,...,...,...
976,2023-01-16 02:18:08.317,4,msedge.exe,0
977,2023-01-16 02:18:08.317,5,Chrome_WidgetWin_1,0
978,2023-01-16 03:19:47.080,3,esrv.exe,0
979,2023-01-16 03:19:47.080,4,VsDebugConsole.exe,0
